<img src="https://upload.wikimedia.org/wikipedia/commons/6/69/XGBoost_logo.png"
     align="right"
     width="40%"/>

# XGBoost for Gradient Boosted Trees

[XGBoost](https://xgboost.readthedocs.io/en/latest/) is a library used for training gradient boosted supervised machine learning models, and it has a [Dask integration](https://xgboost.readthedocs.io/en/latest/tutorials/dask.html) for distributed training. In this guide, you'll learn how to train an XGBoost model in parallel using Dask and Coiled. Download {download}`this jupyter notebook <dask-xgboost.ipynb>` to follow along.

## About the Data

In this example we will use a dataset that joins the
Uber/Lyft dataset from the [High-Volume For-Hire Services](https://www.nyc.gov/site/tlc/businesses/high-volume-for-hire-services.page), with the [NYC Taxi Zone Lookup Table](https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page). 

This results in a dataset with ~1.4 billion rows. 

## Get a Coiled Cluster

To start we need to spin up a Dask cluster.

In [ ]:
%%time

import coiled

cluster = coiled.Cluster(
    n_workers=50,
    name="xgboost",
    worker_vm_types=["r6i.large"],
    scheduler_vm_types=["m6i.large"],
    backend_options={"region": "us-east-2"},
)

client = cluster.get_client()

## Load and Engineer Data

In [ ]:
import dask.dataframe as dd

df = dd.read_parquet(
    "s3://coiled-datasets/dask-xgboost-example/feature_table.parquet/"
)

# Convert dtypes
df = df.astype({
    c: "float32" 
    for c in df.select_dtypes(include="float").columns.tolist()
}).persist()

# Categorize
df = df.categorize(columns=df.select_dtypes(include="category").columns.tolist())

df = df.persist()

## Custom Cross-validation

In [ ]:
def make_cv_splits(n_folds = 5):
    frac = [1 / n_folds] * n_folds
    splits = df.random_split(frac, shuffle=True)
    for i in range(n_folds):
        train = [splits[j] for j in range(n_folds) if j != i]
        test = splits[i]
        yield dd.concat(train), test

## Train Model

When using XGBoost with Dask, we need to call the XGBoost Dask interface from the client side. The main difference with XGBoost’s Dask interface is that we pass our Dask client as an additional argument for carrying out the computation. Note that if the `client` parameter below is set to `None`, XGBoost will use the default client returned by Dask.


In [ ]:
from datetime import datetime

import dask.array as da
import xgboost
from dask_ml.metrics import mean_squared_error

start = datetime.now()
scores = []

for i, (train, test) in enumerate(make_cv_splits(5)):
    print(f"Train/Test split #{i + 1} / 5")
    y_train = train["trip_time"]
    X_train = train.drop(columns=["trip_time"])
    y_test = test["trip_time"]
    X_test = test.drop(columns=["trip_time"])

    d_train = xgboost.dask.DaskDMatrix(None, X_train, y_train, enable_categorical=True)

    print("Training ...")
    model = xgboost.dask.train(
        None,
        {"tree_method": "hist"},
        d_train,
        num_boost_round=4,
        evals=[(d_train, "train")],
    )

    print("Scoring ...")
    predictions = xgboost.dask.predict(None, model, X_test)

    score = mean_squared_error(
        y_test.to_dask_array(),
        predictions.to_dask_array(),
        squared=False,
        compute=False,
    )
    scores.append(score.reshape(1).persist())
    print()
    print("-" * 80)
    print()

scores = da.concatenate(scores).compute()
print(f"RSME={scores.mean()} +/- {scores.std()}")
print(f"Total time:  {datetime.now() - start}")

## Inspect Model

In [ ]:
model

## Clean up

In [ ]:
cluster.close()